## Experiments: Comparing RSAST with other methods

In [797]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST',
 'c:\\ProgramData\\Anaconda3\\python39.zip',
 'c:\\ProgramData\\Anaconda3\\DLLs',
 'c:\\ProgramData\\Anaconda3\\lib',
 'c:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\Nicolas R\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Nicolas R\\.ipython',
 'c:\\Users\\Nicolas R\\random_sast/sast',
 'c:\\Users\\Nicolas R\\random_sast\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST/sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast/cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast\\Experimentation

In [798]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
import matplotlib.pyplot as plt
import os
import seaborn as sns
import math
from utils_sast import delete_files_in_directory

### Reading Datasets

In [799]:

# Set directory where the csv files are located
directory = os.getcwd()+'/results_other_methods/results_by_classifier'

# Create an empty list to store the dataframes
dfs = []

header=['dataset']
res = [str(ele) for ele in range(30)]
header.extend(res)

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, names=header,index_col=False)

        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)


# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)

#10 Resampling
res=res[:10]

df_other_methods["avg"]=df_other_methods[res].mean(axis=1)

df_other_methods=df_other_methods[['filename','dataset','avg']]
df_other_methods['method']=df_other_methods['filename'].str.split('.').str[0]+"_UCR_10RES"
df_other_methods
df_other_methods=df_other_methods.rename(columns={'avg':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3944428890.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3944428890.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3944428890.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3944428890.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,i

In [800]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/results_other_methods/results_summarized/UCR_ARCHIVE_singleTrainTest_ALL_METHODS_MOD.csv'

df = pd.read_csv(filepath)
df = df.melt(id_vars=["Unnamed: 0"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Unnamed: 0':'dataset'})
df['method'] = df['method']+"_UCR_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [801]:
# Set directory where the csv files are located
"""
filepath = os.getcwd()+'/ResultsSummarized/UCR_ARCHIVE_100RESAMPLES_ALL_METHODS_MOD.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Unnamed: 0"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Unnamed: 0':'dataset'})
df['method'] = df['method']+"_UCR_AVG_100RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])
"""

'\nfilepath = os.getcwd()+\'/ResultsSummarized/UCR_ARCHIVE_100RESAMPLES_ALL_METHODS_MOD.csv\'\ndf = pd.read_csv(filepath)\ndf = df.melt(id_vars=["Unnamed: 0"], \n        var_name="method", \n        value_name="score")\ndf = df.rename(columns={\'Unnamed: 0\':\'dataset\'})\ndf[\'method\'] = df[\'method\']+"_UCR_AVG_100RES"\ndf=df[[\'dataset\',\'score\',\'method\']]\ndf_other_methods=pd.concat([df_other_methods,df])\n'

In [802]:

# Set directory where the csv files are located
#df_other_methods=df_other_methods[df_other_methods.method!="STC"]
filepath = os.getcwd()+'/results_other_methods/results_summarized/SAST_PAPER_singleTrainTest_SAST_ST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','STC':'score'})
df['method']="_STC_PAPERSAST_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])



In [803]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/results_other_methods/results_summarized/SAST_PAPER_singleTrainTest_SAST_ST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','0':'score'})
df['method']="_SAST_PAPERSAST_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])

In [804]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/results_other_methods/results_summarized/ROCKET_PAPER_10RESAMPLES_BAKE_OFF.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Dataset"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Dataset':'dataset'})
df['method'] = df['method']+"_PAPERROCKET_AVG_10RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [805]:
# Set directory where the csv files are located


filepath = os.getcwd()+'/results_other_methods/results_summarized/ROCKET_PAPER_10RESAMPLES_2018_DS.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Dataset"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Dataset':'dataset'})
df['method'] = df['method']+"_PAPERROCKET_AVG_10RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [806]:
# Set directory where the csv files are located


filepath = os.getcwd()+'/results_other_methods/results_summarized/RDST_PAPER_30RESAMPLES_ALL_METHODS.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["dataset_name"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'dataset_name':'dataset'})
df['method'] = df['method']+"_PAPERRDST_AVG_30RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [807]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/results_other_methods/results_summarized/RDST_PAPER_30RESAMPLES_RDST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'dataset':'dataset','acc_mean':'score','model':'method'})
df['method'] = df['method']+"_PAPERRDST_AVG_30RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [808]:

# Set directory where the csv files are located
directory = os.getcwd()+'/results_other_methods/results_summarized/Server17_Comparison_Other_Methods_singleTrainTest'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df = pd.concat(dfs, ignore_index=True)
df=df[['dataset_name','classifier_name','accuracy']]

df=pd.pivot_table(df, index=['dataset_name','classifier_name'],aggfunc='mean').reset_index()
df['method']=df['classifier_name'].str.split('_').str[0]+"_RSAST_TRAIN_TEST"
df=df.rename(columns={'dataset_name':'dataset','accuracy':'score'})

df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [809]:
# Set directory where the csv files are located
directory = os.getcwd()+'/results_other_methods/results_summarized/Server17_Comparison_Other_Methods_Resampling'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df = pd.concat(dfs, ignore_index=True)
df=df[['dataset_name','classifier_name','accuracy']]

df=pd.pivot_table(df, index=['dataset_name','classifier_name'],aggfunc='mean').reset_index()
df['method']=df['classifier_name'].str.split('_').str[0]+"_RSAST_10RES"
df=df.rename(columns={'dataset_name':'dataset','accuracy':'score'})

df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [810]:
# Set directory where the RSAST result csv files are located

directories=[]
default_split_acfandpacf_k_1_10_sast=False
default_split_acfandpacf_k_1_10_notsast=False
default_split_all_k_1_10=False
default_split_all_acfandpacf_k_30_50=False
default_split_maxpacf_none_k_30_50=False
default_split_all_acfandpacf=False
default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100=False

resampling_acfandpacf_k_1_10_sast=True

if default_split_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST')
if default_split_acfandpacf_k_1_10_notsast:
    directories.append(os.getcwd()+'/results_rsast/Server17_not_used_sast')
if default_split_all_k_1_10:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_All')   
if default_split_all_acfandpacf_k_30_50:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_Extra_1')    
if default_split_maxpacf_none_k_30_50:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_Extra_2')    
if resampling_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/results_rsast/Server17_Resampling_Comparison_RSAST')     
if default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100:
    directories.append(os.getcwd()+'/results_rsast/Server16_Hyperparameter_Tunning')
    directories.append(os.getcwd()+'/results_rsast/Server17_Hyperparameter_Tunning')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            df['directory']=directory
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3821906961.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3821906961.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3821906961.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\3821906961.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.

In [811]:
hyperparameter='ACF&PACF: n_random_points=10 nb_inst_per_class=10'

In [812]:
df_result=df_result[['accuracy','dataset_name','classifier_name','time']]
df_result=df_result[df_result["classifier_name"]==hyperparameter]
df_result['classifier_name'].replace(hyperparameter,"RSAST",inplace=True)
hyperparameter="RSAST"
df_result.dataset_name.unique()

array(['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY',
       'AllGestureWiimoteZ', 'ArrowHead', 'Beef', 'BeetleFly',
       'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown',
       'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers',
       'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
       'DodgerLoopWeekend', 'Earthquakes', 'ECG200', 'ECG5000',
       'ECGFiveDays', 'ElectricDevices', 'EOGHorizontalSignal',
       'EOGVerticalSignal', 'EthanolLevel', 'FaceAll', 'FaceFour',
       'FacesUCR', 'FiftyWords', 'Fish', 'FordA', 'FordB',
       'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi',
       'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3',
       'GesturePebbleZ1', 'GesturePebbleZ2', 'GunPoint',
       'GunPointAgeSpan', 'GunPointMaleVersusFemale',
       'GunPointOldVersusYoung', 'Ham', 'HandOu

In [813]:
len(df_result.dataset_name.unique())

63

In [814]:
df_other_methods=df_other_methods.rename(columns={"dataset":"dataset_name","score":"accuracy","method":"classifier_name"})
df_other_methods.classifier_name.unique()

array(['BoP_UCR_10RES', 'BOSS_UCR_10RES', 'CID_DTW_UCR_10RES',
       'DDTW_UCR_10RES', 'DD_DTW_UCR_10RES', 'DTD_C_UCR_10RES',
       'DTW_F_UCR_10RES', 'EE_UCR_10RES', 'Flat-COTE_UCR_10RES',
       'FS_UCR_10RES', 'HIVE-COTE_UCR_10RES', 'LPS_UCR_10RES',
       'LS_UCR_10RES', 'MSM_UCR_10RES', 'RotF_UCR_10RES',
       'SAXVSM_UCR_10RES', 'ST_UCR_10RES', 'TSBF_UCR_10RES',
       'TSF_UCR_10RES', 'TWE_UCR_10RES', 'WDTW_UCR_10RES',
       'NB_UCR_TRAIN_TEST', 'C45_UCR_TRAIN_TEST', 'SVML_UCR_TRAIN_TEST',
       'SVMQ_UCR_TRAIN_TEST', 'BN_UCR_TRAIN_TEST', 'RandF_UCR_TRAIN_TEST',
       'RotF_UCR_TRAIN_TEST', 'MLP_UCR_TRAIN_TEST',
       'Euclidean_1NN_UCR_TRAIN_TEST', 'DTW_R1_1NN_UCR_TRAIN_TEST',
       'DTW_Rn_1NN_UCR_TRAIN_TEST', 'DDTW_R1_1NN_UCR_TRAIN_TEST',
       'DDTW_Rn_1NN_UCR_TRAIN_TEST', 'ERP_1NN_UCR_TRAIN_TEST',
       'LCSS_1NN_UCR_TRAIN_TEST', 'MSM_1NN_UCR_TRAIN_TEST',
       'TWE_1NN_UCR_TRAIN_TEST', 'WDDTW_1NN_UCR_TRAIN_TEST',
       'WDTW_1NN_UCR_TRAIN_TEST', 'DD_DTW_UCR_TRA

In [815]:

#Comparison RSAST, SAST and STC (No resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['ST_UCR_TRAIN_TEST','SAST_PAPERSAST_TRAIN_TEST','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and Shapelet Methods (10 resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['R_DST_PAPERRDST_AVG_30RES','FS_UCR_RES_10', 'LS_UCR_RES_10','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and Other Categories (distance, interval, dictionary)(10 resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['EE_UCR_RES_10', 'TSBF_UCR_RES_10','BOSS_UCR_RES_10','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and SOTA methods(10 resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['Rocket_PAPERROCKET_AVG_10RES','HCTE_PAPERROCKET_AVG_10RES','CHIEF_PAPERROCKET_AVG_10RES','DTW_R1_1NN_UCR_TRAIN_TEST'])]

df_other_methods=df_other_methods[df_other_methods['classifier_name'].isin(['RDST_RSAST_10RES','Rocket_RSAST_10RES','Rocket_PAPERROCKET_AVG_10RES','HIVE-COTE_UCR_10RES','HCTE_PAPERROCKET_AVG_10RES'])]
df_other_methods['classifier_name'].unique()

array(['HIVE-COTE_UCR_10RES', 'Rocket_PAPERROCKET_AVG_10RES',
       'HCTE_PAPERROCKET_AVG_10RES', 'RDST_RSAST_10RES',
       'Rocket_RSAST_10RES'], dtype=object)

In [816]:
"""
df_other_methods['classifier_name'].replace("DTW_R1_1NN_UCR_TRAIN_TEST","1NN-DTW",inplace=True)
df_other_methods['classifier_name'].replace("RotF_UCR_TRAIN_TEST","RotF",inplace=True)
df_other_methods['classifier_name'].replace("ST_UCR_TRAIN_TEST","STC",inplace=True)
df_other_methods['classifier_name'].replace("SAST_PAPERSAST_TRAIN_TEST","SAST",inplace=True)
df_other_methods['classifier_name'].replace("FS_UCR_TRAIN_TEST","FS",inplace=True)
df_other_methods['classifier_name'].replace("LS_UCR_TRAIN_TEST","LS",inplace=True)
df_other_methods['classifier_name'].replace("BOSS_UCR_TRAIN_TEST","BOSS",inplace=True)
df_other_methods['classifier_name'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)
df_other_methods['classifier_name'].replace("EE_UCR_TRAIN_TEST","EE",inplace=True)
df_other_methods['classifier_name'].replace("HIVE-COTE_UCR_TRAIN_TEST","HIVE-COTE",inplace=True)
df_other_methods['classifier_name'].replace("Rocket_RSAST_TRAIN_TEST","ROCKET",inplace=True)
df_other_methods['classifier_name'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)
df_other_methods['classifier_name'].replace("ResNet_PAPERROCKET_AVG_10RES","ResNet",inplace=True)
df_other_methods['classifier_name'].replace("HCTE_PAPERROCKET_AVG_10RES","HIVE-COTE",inplace=True)
df_other_methods['classifier_name'].replace("ITime_PAPERROCKET_AVG_10RES","InceptionTime",inplace=True)
df_other_methods['classifier_name'].replace("CHIEF_PAPERROCKET_AVG_10RES","TS-CHIEF",inplace=True)
df_other_methods['classifier_name'].replace("Rocket_PAPERROCKET_AVG_10RES","ROCKET_ORIG",inplace=True)
df_other_methods['classifier_name'].replace("BOSS_PAPERROCKET_AVG_10RES","BOSS",inplace=True)
df_other_methods['classifier_name'].replace("ST_PAPERROCKET_AVG_10RES","ST",inplace=True)
df_other_methods['classifier_name'].replace("R_DST_PAPERRDST_AVG_30RES","RDST",inplace=True)
df_other_methods['classifier_name'].replace("FS_UCR_RES_10","FS",inplace=True)
df_other_methods['classifier_name'].replace("LS_UCR_RES_10","LS",inplace=True)
df_other_methods['classifier_name'].replace("BOSS_UCR_RES_10","BOSS",inplace=True)
df_other_methods['classifier_name'].replace("EE_UCR_RES_10","EE",inplace=True)
df_other_methods['classifier_name'].replace("TSBF_UCR_RES_10","TSBF",inplace=True)
df_other_methods['classifier_name'].replace("RDST_RSAST_10RES","RDST",inplace=True)
df_other_methods['classifier_name'].replace("Rocket_RSAST_10RES","ROCKET",inplace=True)
"""

'\ndf_other_methods[\'classifier_name\'].replace("DTW_R1_1NN_UCR_TRAIN_TEST","1NN-DTW",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("RotF_UCR_TRAIN_TEST","RotF",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("ST_UCR_TRAIN_TEST","STC",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("SAST_PAPERSAST_TRAIN_TEST","SAST",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("FS_UCR_TRAIN_TEST","FS",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("LS_UCR_TRAIN_TEST","LS",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("BOSS_UCR_TRAIN_TEST","BOSS",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("EE_UCR_TRAIN_TEST","EE",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("HIVE-COTE_UCR_TRAIN_TEST","HIVE-COTE",inplace=True)\ndf_other_methods[\'classifier_name\'].replace("Rocket_RSAST_TRAIN_TEST","ROCKET",inplac

In [817]:
df_other_methods['classifier_name'].unique()

array(['HIVE-COTE_UCR_10RES', 'Rocket_PAPERROCKET_AVG_10RES',
       'HCTE_PAPERROCKET_AVG_10RES', 'RDST_RSAST_10RES',
       'Rocket_RSAST_10RES'], dtype=object)

In [818]:
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)


In [819]:
concatenated_df['classifier_name'].unique()

array(['HIVE-COTE_UCR_10RES', 'Rocket_PAPERROCKET_AVG_10RES',
       'HCTE_PAPERROCKET_AVG_10RES', 'RDST_RSAST_10RES',
       'Rocket_RSAST_10RES', 'RSAST'], dtype=object)

In [820]:
len(concatenated_df.dataset_name.unique())

150

### Overall analysis

#### Accuracy: RSAST vs Other methods

In [821]:
max_bx=max(df_result.accuracy)
min_bx=min(df_result.accuracy)


#df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
#df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])

#print(df_to_cd.head(5))

# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=df_result, y='dataset_name', x='accuracy')
#plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

#plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
#print(df_to_cd.describe())
#plt.tight_layout()
# save plot
plt.savefig('images_boxplot_acc/boxplot_acc_per_ds.png')

C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_12348\853197985.py:17: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')


In [822]:
filter_param=concatenated_df[concatenated_df.classifier_name==hyperparameter]

In [823]:
stats = filter_param.groupby(['dataset_name'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []
ci95_hi_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))
    ci95_hi_lo.append(1.96*s/math.sqrt(c))

#stats['ci95_hi'] = ci95_hi
#stats['ci95_lo'] = ci95_lo
stats['ci95_interval'] = ci95_hi_lo
print(stats.round(2))

                       mean  count   std  ci95_interval
dataset_name                                           
ACSF1                  0.29     10  0.02           0.01
Adiac                  0.63     10  0.01           0.01
AllGestureWiimoteX     0.67     10  0.01           0.00
AllGestureWiimoteY     0.71     10  0.01           0.01
AllGestureWiimoteZ     0.63     10  0.01           0.01
...                     ...    ...   ...            ...
Herring                0.59     10  0.02           0.01
MelbournePedestrian    0.87     10  0.00           0.00
PLAID                  0.81     10  0.01           0.01
PickupGestureWiimoteZ  0.71     10  0.01           0.01
ShakeGestureWiimoteZ   0.80     10  0.01           0.01

[63 rows x 4 columns]


In [824]:
# create a pivot table with the mean of score by hyperparameter
t_score_by_method=pd.pivot_table(concatenated_df, values='accuracy', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')
#t_score_by_method=score_by_method.transpose()[[hyperparameter,"SAST"]].round(2)
#t_score_by_method=score_by_method.transpose().round(2)
t_score_by_method.iloc[0:25,:]


classifier_name,HCTE_PAPERROCKET_AVG_10RES,HIVE-COTE_UCR_10RES,RDST_RSAST_10RES,RSAST,Rocket_PAPERROCKET_AVG_10RES,Rocket_RSAST_10RES
dataset_name,,,,,,
ACSF1,NaN,NaN,0.768000,0.292000,0.8860,0.766000
Adiac,0.8107,0.809207,0.754987,0.625064,0.7834,0.767519
AllGesWiiX,NaN,NaN,NaN,NaN,0.7900,NaN
AllGesWiiY,NaN,NaN,NaN,NaN,0.7727,NaN
AllGesWiiZ,NaN,NaN,NaN,NaN,0.7661,NaN
AllGestureWiimoteX,NaN,NaN,0.731714,0.669714,NaN,0.760000
AllGestureWiimoteY,NaN,NaN,0.802857,0.709286,NaN,0.812429
AllGestureWiimoteZ,NaN,NaN,0.680286,0.630857,NaN,0.689429
ArrowHead,0.8629,0.897143,0.952000,0.780000,0.8143,0.911429


In [825]:
t_score_by_method.iloc[25:49,:]

classifier_name,HCTE_PAPERROCKET_AVG_10RES,HIVE-COTE_UCR_10RES,RDST_RSAST_10RES,RSAST,Rocket_PAPERROCKET_AVG_10RES,Rocket_RSAST_10RES
dataset_name,,,,,,
DistalPhalanxOutlineAgeGroup,0.7626,0.820863,0.808633,0.764029,0.7590,0.825180
DistalPhalanxOutlineCorrect,0.7717,0.814130,0.822464,0.784783,0.7696,0.843841
DistalPhalanxTW,0.6835,0.705036,0.716547,0.742446,0.7187,0.720863
DodgerLoopDay,NaN,NaN,0.542500,0.482500,0.5725,0.590000
DodgerLoopGame,NaN,NaN,0.859420,0.779710,0.8732,0.871014
DodgerLoopWeekend,NaN,NaN,0.978261,0.972464,0.9746,0.978261
ECG200,0.8500,0.879000,0.879000,0.865000,0.9060,0.869000
ECG5000,0.9462,0.946600,0.950133,0.937067,0.9472,0.948556
ECGFiveDays,1.0000,0.994077,0.994774,0.985947,1.0000,0.999652


In [826]:
t_score_by_method.iloc[49:75,:]

classifier_name,HCTE_PAPERROCKET_AVG_10RES,HIVE-COTE_UCR_10RES,RDST_RSAST_10RES,RSAST,Rocket_PAPERROCKET_AVG_10RES,Rocket_RSAST_10RES
dataset_name,,,,,,
FreezerSmallTrain,NaN,NaN,0.995474,0.847789,0.9496,0.996035
Fungi,NaN,NaN,0.634409,0.591398,1.0000,0.634409
GestureMidAirD1,NaN,NaN,0.738462,0.646154,0.7169,0.695385
GestureMidAirD2,NaN,NaN,0.569231,0.530769,0.6608,0.628462
GestureMidAirD3,NaN,NaN,0.353077,0.353846,0.4146,0.353846
GesturePebbleZ1,NaN,NaN,0.984884,0.934302,0.9058,0.968023
GesturePebbleZ2,NaN,NaN,0.974684,0.937342,0.8304,0.966456
GunPoint,1.0000,0.998667,0.993333,0.979333,1.0000,0.995333
GunPointAgeSpan,NaN,NaN,0.993354,0.969937,0.9968,0.990506


In [827]:
t_score_by_method.iloc[76:,:]

classifier_name,HCTE_PAPERROCKET_AVG_10RES,HIVE-COTE_UCR_10RES,RDST_RSAST_10RES,RSAST,Rocket_PAPERROCKET_AVG_10RES,Rocket_RSAST_10RES
dataset_name,,,,,,
Mallat,0.9620,0.973092,0.947420,NaN,0.9559,0.925842
Meat,0.9333,0.983333,1.000000,NaN,0.9483,1.000000
MedicalImages,0.7776,0.807763,0.800526,NaN,0.7995,0.832895
MelbournePedestrian,NaN,NaN,0.955966,0.865929,0.9044,0.901271
MiddlePhalanxOutlineAgeGroup,0.5974,0.688312,0.795455,NaN,0.5903,0.762338
...,...,...,...,...,...,...
Wine,0.7778,0.888889,0.981481,NaN,0.8130,0.912963
WordSynonyms,0.7382,0.745768,0.756740,NaN,0.7534,0.747806
Worms,0.5584,0.732468,0.687013,NaN,0.7403,0.740260


In [828]:
t_score_by_method.describe().round(2)

classifier_name,HCTE_PAPERROCKET_AVG_10RES,HIVE-COTE_UCR_10RES,RDST_RSAST_10RES,RSAST,Rocket_PAPERROCKET_AVG_10RES,Rocket_RSAST_10RES
count,85.00,85.00,128.00,63.00,128.00,128.00
mean,0.85,0.87,0.86,0.79,0.85,0.86
std,0.14,0.13,0.15,0.17,0.16,0.15
min,0.38,0.38,0.32,0.29,0.10,0.27
25%,0.77,0.81,0.78,0.68,0.78,0.78
50%,0.86,0.90,0.91,0.81,0.90,0.90
75%,0.96,0.97,0.98,0.94,0.97,0.97
max,1.00,1.00,1.00,1.00,1.00,1.00


In [829]:
t_score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=t_score_by_method.transpose()#[[hyperparameter,"SAST"]].round(2)

In [830]:
#wins of RSAST
#t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method["SAST"].round(2))].index

In [831]:
#concatenated_df['classifier_name']=concatenated_df['classifier_name'].str.split('_').str[0]

In [832]:
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'], aggfunc='var').round(4)

,accuracy
classifier_name,
HCTE_PAPERROCKET_AVG_10RES,0.0195
HIVE-COTE_UCR_10RES,0.0161
RDST_RSAST_10RES,0.0213
RSAST,0.0283
Rocket_PAPERROCKET_AVG_10RES,0.0253
Rocket_RSAST_10RES,0.0211


In [833]:
pv_all=pd.pivot_table(concatenated_df,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
concatenated_df=concatenated_df[concatenated_df.dataset_name.isin(ds_complete)]

In [834]:
#concatenated_df=concatenated_df[concatenated_df.dataset_name!="Fungi"]
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'], aggfunc='count')

,accuracy
classifier_name,
HCTE_PAPERROCKET_AVG_10RES,35
HIVE-COTE_UCR_10RES,35
RDST_RSAST_10RES,35
RSAST,35
Rocket_PAPERROCKET_AVG_10RES,35
Rocket_RSAST_10RES,35


In [835]:
#concatenated_df['classifier_name']=concatenated_df['classifier_name'].str.split('_').str[0]
score_by_method=pd.pivot_table(df_perf, values='accuracy', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')
score_by_method

classifier_name,HCTE_PAPERROCKET_AVG_10RES,HIVE-COTE_UCR_10RES,RDST_RSAST_10RES,RSAST,Rocket_PAPERROCKET_AVG_10RES,Rocket_RSAST_10RES
dataset_name,,,,,,
Adiac,0.8107,0.809207,0.754987,0.625064,0.7834,0.767519
ArrowHead,0.8629,0.897143,0.952000,0.780000,0.8143,0.911429
Beef,0.9333,0.763333,0.620000,0.480000,0.8333,0.746667
BeetleFly,0.9500,0.965000,0.900000,0.850000,0.9000,0.850000
BirdChicken,0.8500,0.930000,0.950000,0.815000,0.9000,0.965000
CBF,0.9989,0.999111,0.999111,0.982333,1.0000,0.992111
Car,0.8667,0.918333,0.933333,0.920000,0.8467,0.921667
ChlorineConcentration,0.7120,0.725260,0.775651,0.751589,0.8145,0.813984
CinCECGTorso,0.9964,0.991014,0.991087,0.929420,0.8362,0.877246


In [836]:
#hyperparameter='RDST_30RES'

In [837]:
t_score_by_method=pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=t_score_by_method.transpose()#[[hyperparameter,"SAST"]].round(2)
for methods in t_score_by_method.columns.drop(hyperparameter):
    print("method: "+methods+" vs "+hyperparameter)
    total_wins=len(t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("winning "+hyperparameter+": "+str(total_wins))
    total_loss=len(t_score_by_method[0>(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("losse "+hyperparameter+": "+str(total_loss))
    total_draws=len(t_score_by_method[0==(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("draws: "+str(total_draws))

method: HCTE_PAPERROCKET_AVG_10RES vs RSAST
winning RSAST: 11
losse RSAST: 21
draws: 3
method: HIVE-COTE_UCR_10RES vs RSAST
winning RSAST: 5
losse RSAST: 27
draws: 3
method: RDST_RSAST_10RES vs RSAST
winning RSAST: 2
losse RSAST: 29
draws: 4
method: Rocket_PAPERROCKET_AVG_10RES vs RSAST
winning RSAST: 10
losse RSAST: 22
draws: 3
method: Rocket_RSAST_10RES vs RSAST
winning RSAST: 4
losse RSAST: 27
draws: 4


In [838]:
delete_files_in_directory("images_one_vs_one_comparison/")
for method in score_by_method.columns.drop(hyperparameter):
    y=score_by_method[hyperparameter]
    x=score_by_method[method]

    # set the style of the plot
    plt.style.use('seaborn')

    # create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # plot the line
    ax.plot([0,1], [0,1])

    # plot the points
    ax.scatter(x, y, color='blue', edgecolor='blue', s=30, zorder=10)

    # add labels and title
    ax.set_ylabel(hyperparameter)
    ax.set_xlabel(method)
    ax.set_title('Accuracy')


    # set the background color
    ax.set_facecolor('lightgrey')
    plt.savefig('images_one_vs_one_comparison/'+hyperparameter+'vs'+method+'.png')


All files deleted successfully.


In [839]:
# create a pivot table with the mean of time by hyperparameter
time_per_method=pd.pivot_table(df_result, values='time', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')/60
time_per_method

classifier_name,RSAST
dataset_name,
ACSF1,76.067852
Adiac,0.404516
AllGestureWiimoteX,1.693926
AllGestureWiimoteY,1.599983
AllGestureWiimoteZ,1.646046
...,...
Herring,0.395759
MelbournePedestrian,0.019043
PLAID,71.267501


#### Generate CD Diagram comparison

In [840]:
concatenated_df.classifier_name.unique()

array(['HIVE-COTE_UCR_10RES', 'Rocket_PAPERROCKET_AVG_10RES',
       'HCTE_PAPERROCKET_AVG_10RES', 'RDST_RSAST_10RES',
       'Rocket_RSAST_10RES', 'RSAST'], dtype=object)

In [841]:

from cd_function import *
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, fname='images_cd_diagram/cd-diagram_overall_comparison_other_method.png')


{0: 'RSAST', 1: 'HCTE_PAPERROCKET_AVG_10RES', 2: 'Rocket_PAPERROCKET_AVG_10RES', 3: 'HIVE-COTE_UCR_10RES', 4: 'Rocket_RSAST_10RES', 5: 'RDST_RSAST_10RES'}
[1, 2, 0]
[1, 2, 3, 4, 5]


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [842]:
df_perf["classifier_name"].unique()

array(['HCTE_PAPERROCKET_AVG_10RES', 'HIVE-COTE_UCR_10RES',
       'RDST_RSAST_10RES', 'RSAST', 'Rocket_PAPERROCKET_AVG_10RES',
       'Rocket_RSAST_10RES'], dtype=object)

In [843]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
RSAST,4.885714
HCTE_PAPERROCKET_AVG_10RES,3.900000
Rocket_PAPERROCKET_AVG_10RES,3.485714
HIVE-COTE_UCR_10RES,3.457143
Rocket_RSAST_10RES,2.842857
RDST_RSAST_10RES,2.428571


In [844]:
uca_ds=pd.read_excel("DataSetsUCLASummary.xlsx")

uca_ds.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'N RUNS S17_SAST_DS_all', 'N RUNS S17_SAST_DS_extra_1',
       'N RUNS S17_SAST_DS_extra_2', 'N RUNS S17_RDST_ROCKET',
       'N RUNS S17_RSAST_R10', 'ID', 'Name', 'Type', 'Train ', 'Test ',
       'Class', 'Length', 'RSAST (10,10) WORST SAST?', 'O(|c|nm²)',
       'O(|c|nm³)', 'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 29'],
      dtype='object')

In [845]:
concatenated_df=concatenated_df.merge(uca_ds, left_on=['dataset_name'],right_on=['Name'])
concatenated_df.columns

Index(['dataset_name', 'accuracy', 'classifier_name', 'time', 'USED SAST',
       'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'N RUNS S17_SAST_DS_all', 'N RUNS S17_SAST_DS_extra_1',
       'N RUNS S17_SAST_DS_extra_2', 'N RUNS S17_RDST_ROCKET',
       'N RUNS S17_RSAST_R10', 'ID', 'Name', 'Type', 'Train ', 'Test ',
       'Class', 'Length', 'RSAST (10,10) WORST SAST?', 'O(|c|nm²)',
       'O(|c|nm³)', 'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 29'],
      dtype='object')

In [846]:
acc_per_type=pd.pivot_table(concatenated_df, values="accuracy", index=["dataset_name","Type",'classifier_name'])
#acc_per_type=concatenated_df
acc_per_type=acc_per_type.reset_index()

In [847]:
max_bx=max(acc_per_type.accuracy)
min_bx=min(acc_per_type.accuracy)


#df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
#df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])

#print(df_to_cd.head(5))

# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=acc_per_type, y='Type', x='accuracy', hue ='classifier_name')
#plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

#plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
#print(df_to_cd.describe())
plt.tight_layout()
# save plot
plt.savefig('images_boxplot_acc_per_type/boxplot_acc_per_type.png')

In [848]:
pd.pivot_table(acc_per_type,index='Type', aggfunc=lambda x: len(x.unique()))

,accuracy,classifier_name,dataset_name
Type,,,
Device,12,6,2
ECG,17,6,3
Image,84,6,15
Motion,29,6,5
Sensor,35,6,6
Simulated,6,6,1
Spectro,14,6,3
